# Year 2023 Day 19

The goal of this notebook is multiple:

1. Learn how to use use Sankey diagrams with `plotly`
2. Visualize the problem's input
3. Describe the problem solving procedure


In [ ]:
from dataclasses import dataclass
from typing import Literal

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xarray as xr

from advent_of_code.common.common import get_example_inputs_file_contents
from advent_of_code.visualization.plotly import (
    ValuedLink,
    build_sankey_figure,
    to_plotly_sankey_input,
)
from advent_of_code.y_2023.problem_202311 import (
    create_coord_array,
)
from advent_of_code.y_2023.problem_202319 import (
    AdventOfCodeProblem202319,
    PartRatingRange,
    PartRatingRangeTree,
    construct_initial_part_range,
)

## Make a Sankey Diagram with Plotly

First, let's start with the example provided in
[Plotly: Sankey Diagram](https://plotly.com/python/sankey-diagram/).

Then, the goal is to adapt the in-memory representation of the problem's data into something that Plotly can understand and plot.

First, a builder can be made for those Plotly diagrams, then, an object converter to convert the in-memory representation of the problem's data into this intermediate data structure.

One challenge will be to flatten the tree structure to a list of weighted edges (flat graph representation).

The `source, target, value` expected by plotly indicates the need to flatten any tree structure.

In the following, the word "link" means a graph's directional edge/vertex


In [ ]:
@dataclass(frozen=True)
class ValuedLink:
    source: str
    target: str
    value: int


@dataclass(frozen=True)
class PlotlySankeyInput:
    labels: list[str]
    sources: list[int]
    targets: list[int]
    values: list[int]
    node_colors: list[str]
    link_colors: list[str]


def to_mapping_label_to_index(valued_links: list[ValuedLink]) -> dict[str, int]:
    collected_ordered_set: dict[str, None] = {}
    for link in valued_links:
        collected_ordered_set[link.source] = None
        collected_ordered_set[link.target] = None
    collected = {k: idx for idx, k in enumerate(collected_ordered_set)}
    return collected


def to_plotly_sankey_input(
    valued_links: list[ValuedLink],
    *,
    color_mode: Literal["random", "problem_202319"] = "random",
) -> PlotlySankeyInput:
    mapping_label_to_index = to_mapping_label_to_index(valued_links)

    sources = []
    targets = []
    values = []

    for link in valued_links:
        source_idx = mapping_label_to_index[link.source]
        target_idx = mapping_label_to_index[link.target]
        value = link.value
        sources.append(source_idx)
        targets.append(target_idx)
        values.append(value)

    labels = list(mapping_label_to_index)
    if color_mode == "random":
        random_color_array = np.random.randint(256, size=(len(sources), 3))
        node_colors = [f"rgb({r}, {g}, {b})" for (r, g, b) in random_color_array]
        link_colors = [f"rgba({r}, {g}, {b}, 0.3)" for (r, g, b) in random_color_array]
    elif color_mode == "problem_202319":
        mapping = {"A": "rgba(0,255,0,0.3)", "R": "rgba(255,0,0,0.3)"}
        node_colors = [mapping.get(node, "rgba(128,128,128,0.3)") for node in labels]
        mapping = {labels.index(k): v for k, v in mapping.items()}
        link_colors = [
            mapping.get(target, "rgba(128,128,128,0.3)") for target in targets
        ]

    else:
        raise NotImplementedError
    return PlotlySankeyInput(labels, sources, targets, values, node_colors, link_colors)


def build_sankey_figure(sankey_input: PlotlySankeyInput) -> go.Figure:
    node_style_dict = dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        color=sankey_input.node_colors,
    )
    fig = go.Figure(
        data=[
            go.Sankey(
                node={**dict(label=sankey_input.labels), **node_style_dict},
                link=dict(
                    source=sankey_input.sources,
                    target=sankey_input.targets,
                    value=sankey_input.values,
                    color=sankey_input.link_colors,
                ),
            )
        ]
    )

    fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
    return fig

In [ ]:
valued_links = [
    ValuedLink("a", "b", 2),
    ValuedLink("b", "c", 5),
    ValuedLink("a", "c", 4),
    ValuedLink("z", "c", 1),
]

sankey_input = to_plotly_sankey_input(valued_links)
display(sankey_input)
build_sankey_figure(sankey_input)

In [ ]:
problem = AdventOfCodeProblem202319()
problem

In [ ]:
example_input = get_example_inputs_file_contents(2023)["test_problem_202319"][
    "EXAMPLE_INPUT"
]
print(example_input)

In [ ]:
puzzle_input = problem.parse_text_input(example_input)
puzzle_input

In [ ]:
puzzle_input.workflows

In [ ]:
initial_part_rating_range = construct_initial_part_range()
initial_part_rating_range

In [ ]:
range_tree = puzzle_input.apply_to_range(initial_part_rating_range)
range_tree

In [ ]:
for label, rating_range_list in range_tree.mapping.items():
    print(label, rating_range_list)

In [ ]:
from dataclasses import asdict
import json
from pathlib import Path

json_txt = json.dumps(asdict(range_tree), indent=4, default=str)
# Path("202319.out.json").write_text(json_repr)

In [ ]:
print(json_repr)

In [ ]:
json_dict = json.loads(json_txt)

In [ ]:
from IPython.display import Markdown as md


def pretty_print_dict_with_json(json_string: str) -> md:
    json_repr = json.dumps(json_string, indent=4)
    return md(
        f"""
```json
{json_repr} 
```
"""
    )

Issue: how to find a good way to get a scalar from the "PartTreeRange" objects?
The cubes are 4D, composed of 4 components x, m, a, s.

Example: The flow coming from `rfg` must be split in three, to `gd`, `R` and `A`, with a metric that correctly reflect the proportion of parts flowing.


In [ ]:
pretty_print_dict_with_json(
    json_dict["children"]["px"][0]["children"]["rfg"][0]["mapping"]
)

A first approach is just to compute the volume of the contiguous cubic sub-space delimitated of the $4000^4$ space formed by the dimensions (x, m, a, s).
Luckily, `plotly` will automatically normalize all the weights associated to edges going into the same node, concretely, incoming flows of `(6, 4, 10)` will visually respectively contribute to 30%, 20%, 50%.


In [ ]:
{
    child: rating_ranges[0].volume()
    for child, rating_ranges in range_tree.children["px"][0]
    .children["rfg"][0]
    .mapping.items()
}

But what to do in case of split flows, when a workflow has multiple subspaces (list of `PartRatingRange` has more than one element)?

Example:


In [ ]:
pretty_print_dict_with_json(
    json_dict["children"]["px"][0]["children"]["rfg"][0]["children"]["gd"][0]
)

In that case, just add the individual contribution. The subspace associated to the workflow is just non contiguous, composed of several cubes.

Below is the code snippet to compute the volumes:


In [ ]:
def compute_flow_values(
    mapping: dict[str, list[PartRatingRange]],
) -> dict[str, tuple[int, ...]]:
    return {
        child: tuple(rating_range.volume() for rating_range in rating_ranges)
        for child, rating_ranges in mapping.items()
    }


compute_flow_values(range_tree.children["px"][0].children["rfg"][0].mapping)

In [ ]:
compute_flow_values(
    range_tree.children["px"][0].children["rfg"][0].children["gd"][0].mapping
)

Now, that the structure can be converted to a tree with integer leaves (the volumes), the Sankey Diagram can be dealt, by iterating depth-first into the `PartRatingRangeTree` structure, and filling a list of `ValuedLink`.


In [ ]:
def to_valued_links(
    tree: PartRatingRangeTree,
    source: str,
    valued_links: list[ValuedLink],
    *,
    sum_flows: bool = False,
) -> None:
    flow_values = compute_flow_values(tree.mapping)
    if sum_flows:
        flow_total_values = {k: sum(v) for k, v in flow_values.items()}
        for target, value in flow_total_values.items():
            valued_links.append(ValuedLink(source, target, value))
    else:
        # More details as multiple flows from same source and target are showed explicitly
        # instead of merged. For the example data, this shows for the two links from lnx to A.
        for target, values in flow_values.items():
            for value in values:
                valued_links.append(ValuedLink(source, target, value))

    children = tree.children

    if children is not None:
        for child_label, child_trees in children.items():
            for child_tree in child_trees:
                to_valued_links(child_tree, child_label, valued_links)


valued_links = []
to_valued_links(range_tree, "in", valued_links)
valued_links

Now that we have got a list of `ValuedLink`, the same conversion function used earlier can be used to feed it into Plotly!

In [ ]:
sankey_input = to_plotly_sankey_input(valued_links, color_mode="problem_202319")
display(sankey_input)
build_sankey_figure(sankey_input)

The result looks similar to this post on Reddit: [\[2023 Day 19 (part 2)\] Sankey diagrams are cool](https://www.reddit.com/r/adventofcode/comments/18lyvuv/2023_day_19_part_2_sankey_diagrams_are_cool/)


## Open Points


In all cases, the projection from a 4D subspace associated to a workflow, composed of one or more small cubes, leads to a loss of "complexity" in the visual representation. Only a 1D metric is used to represent the workflow contributions.

Representing the subspaces would be another challenge in its own. At first, making a version of the code that only sort pieces according to 2 dimensions would help to visualize. For that, some dimensions can be fixed (slicing the 4D cube into 2D). It would be interesting to build such a datacube with `xarray` ; it would have 4 dimensions: `dims = ('x', 'm', 'a', 's')` and a shape of `(4000, 4000, 4000, 4000)`. Slicing it would be easy, eg `xda.isel(a=0, s=0)`.
Since the array would be large, only portions of it could be computed. How to parallelize the algorithm, allowing it to be executed only on a sub-space, eg `(:200, :200, :200, :200)`? Is it possible?
